In [1]:
import os

In [2]:
from src.download_data import download_data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import altair as alt
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
download_data("data", "raw_data.csv")
white_wine_df = pd.read_csv("data/raw_data.csv", index_col = 0)

# Data splitting

In [4]:
train_df, test_df = train_test_split(white_wine_df, test_size=0.25, random_state=123)

X_train = train_df.drop(columns=['quality'])
y_train = train_df['quality']

X_test = test_df.drop(columns=['quality'])
y_test = test_df['quality']

# Preprocessing and pipeline

In [5]:
numeric_features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
                    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

numeric_transformer = make_pipeline(StandardScaler())

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ]
)

# Cross-validation on different models

In [6]:
results_dict = {}

In [7]:
models = {
     
    "dummyregressor": DummyRegressor(),
    "ridge": Ridge(),
    "randomforest": RandomForestRegressor(),

}

scoring={
        "neg_mean_squared_error": "neg_mean_squared_error",
        "neg_root_mean_square": "neg_root_mean_squared_error",
        "neg_mean_absolute_error": "neg_mean_absolute_error",
        "r2": "r2"
     }

for model_name, model in models.items():
    pipe = make_pipeline(preprocessor, model)
    scores = cross_validate(pipe, X_train, y_train, return_train_score= True, scoring = scoring)
    scores_df = pd.DataFrame(scores).mean()
    results_dict[model_name] = scores_df

In [8]:
pd.DataFrame(results_dict)

,dummyregressor,ridge,randomforest
fit_time,0.025926,0.027457,2.637173
score_time,0.017672,0.010495,0.038459
test_neg_mean_squared_error,-0.789925,-0.579467,-0.393573
train_neg_mean_squared_error,-0.789685,-0.568744,-0.055570
test_neg_root_mean_square,-0.888473,-0.761035,-0.627022
train_neg_root_mean_square,-0.888624,-0.754138,-0.235732
test_neg_mean_absolute_error,-0.676655,-0.590999,-0.456513
train_neg_mean_absolute_error,-0.676591,-0.587174,-0.170395
test_r2,-0.000760,0.265500,0.501402
train_r2,0.000000,0.279763,0.929621


We chose `DummyRegressor` as the baseline here. `Ridge` and `RandomForestRegressor` all preform better than the base line `DummyRegressor`. Moreover, we observe that `RandomForestRegressor` seems to be a better model by comparing `Ridge` since we got bigger negative mean squared error, bigger negative root mean squared error, bigger negative mean absolute error and bigger r2 score in the `RandomForestRegressor` model. Therefore, for predicting the test set, we will use `RandomForestRegressor` model, but we still need to investigate more in the future.

# Hyperparameter optimazation with RandomForestRegressor

In [9]:
pipe_randomforest = make_pipeline(preprocessor, RandomForestRegressor(random_state=2020))

param_grid = {
    "randomforestregressor__n_estimators": [10, 50, 100, 150, 200, 250, 300],
    "randomforestregressor__max_depth": [3,5,7, 10]
}

random_search = RandomizedSearchCV(
    pipe_randomforest, 
    param_distributions=param_grid,
    n_iter=28,
    cv=3,
    n_jobs=-1,
    random_state=2020
)




In [10]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['fixed '
                                                                                'acidity',
                                                                                'volatile '
                                                                                'acidity',
                                                                                'citric '
                                                                                'acid',
                                                  

In [11]:
random_search.best_params_

{'randomforestregressor__n_estimators': 300,
 'randomforestregressor__max_depth': 10}

# Final results by using RandomForestRegressor

In [12]:
print("Random Search best model score: " + str(random_search.best_score_))
print("Train score on the train set: " + str(random_search.score(X_train, y_train)))
print("Test score on the test set: " + str(random_search.score(X_test, y_test)))

Random Search best model score: 0.43738839475956953
Train score on the train set: 0.7260837603778569
Test score on the test set: 0.4311472751504126


From above, we can see that the final test score is around 0.4311 which seems to be not reasonable here. Therefore, `RandomForestregressor` may not be a good model to use here. However, we can find other models to improve our test scores.

# A bar chart of feature importances

In [13]:
random_search.best_estimator_


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['fixed acidity',
                                                   'volatile acidity',
                                                   'citric acid',
                                                   'residual sugar',
                                                   'chlorides',
                                                   'free sulfur dioxide',
                                                   'total sulfur dioxide',
                                                   'density', 'pH', 'sulphates',
                                                   'alcohol'])])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=10, n_

In [14]:
importance = random_search.best_estimator_.named_steps["randomforestregressor"].feature_importances_



In [15]:
df = pd.DataFrame(
    {
        "target_feats": X_train.columns,
        "target_weights": importance.transpose()
    })


In [16]:
alt.Chart(df).mark_bar().encode(x = alt.X("target_feats", sort = '-y', title = "target features"),
                                y = alt.Y("target_weights", title = "target weights")
                               ).properties(width = 400, height = 400
                                           ).configure_axis(labelFontSize = 16
)

alt.Chart(...)